# Transfer learning with YoloV5 

In [1]:
import os 
import glob
import numpy as np 
import shutil 

In [2]:
# Create a folder structure for YOLOv5 training
path_to_data = '../output/YOLO_DATA'
path_to_formatted_data = '../output/YOLO_DATA_FORMATTED'
if not os.path.exists(path_to_formatted_data) and os.path.exists(path_to_data):
    for folder in ['images', 'labels']:
        for split in ['train', 'val', 'test']:
            os.makedirs(f'{path_to_formatted_data}/{folder}/{split}')

In [3]:
# Move assets to the correct folder 
def get_filenames(folder,extension='.jpg'):
    filenames = set()
    for path in glob.glob(os.path.join(folder, f'*{extension}')):
        # Extract the filename
        filename = os.path.split(path)[-1]        
        filenames.add(filename)
    return filenames
    
test_images = get_filenames(f"{path_to_data}/TEST/images")
validate_images = get_filenames(f"{path_to_data}/VALIDATE/images")
train_images = get_filenames(f"{path_to_data}/TRAIN/images")

In [4]:
print('Check if there are duplicate images')
print(f'{len(validate_images & test_images)}')
print(f'{len(train_images & test_images)}')
print(f'{len(validate_images & train_images)}')

Check if there are duplicate images
0
0
0


In [5]:
# add data to the correct directory format
for dir in ['TRAIN','TEST','VALIDATE']:
    source_dir = f"{path_to_data}/{dir}"
    image_files = get_filenames(f'{source_dir}/images/')
    for file in image_files:
        if dir == 'VALIDATE':
            dir = 'val'
        shutil.copy(f"{source_dir}/images/{file}", f"{path_to_formatted_data}/images/{dir.lower()}/{file}")
        file = file.replace('.jpg','.txt')
        shutil.copy(f"{source_dir}/labels/{file}", f"{path_to_formatted_data}/labels/{dir.lower()}/{file}")


In [6]:
import torch
use_cuda = torch.cuda.is_available()
use_cuda

True

In [14]:
!python ../yolov5/train.py --data config.yaml --weights yolov5s.pt --epochs 100 --batch 4 --freeze 10

2024-01-07 18:47:30.682291: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 18:47:30.682359: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 18:47:30.682376: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=config.yaml, hyp=../yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8

In [15]:
# tensorboard --logdir ../yolov5/runs/ --host localhost # --bind_all 
# notebook.display(port=6006, height=1000)

In [16]:
!python ../yolov5/val.py --data config.yaml --weights ../yolov5/runs/train/exp/weights/best.pt

val: data=config.yaml, weights=['../yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=../yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-253-g63555c8 Python-3.11.6 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /home/jovyan/output/YOLO_DATA_FORMATTED/labels/val.cache... 71 ima
                 Class     Images  Instances          P          R      mAP50   
                   all         71         99      0.805      0.828      0.818      0.623
Speed: 0.7ms pre-process, 6.2ms inference, 1.3ms NMS per image at shape (32, 3, 640, 640)
Results saved to ../yolov5/runs/val/exp


In [17]:
!python ../yolov5/detect.py --weights ../yolov5/runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source ./test_images**

detect: weights=['../yolov5/runs/train/exp/weights/best.pt'], source=./test_images, data=../yolov5/data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-253-g63555c8 Python-3.11.6 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/9 /home/jovyan/notebooks/test_images/test_1.jpg: 320x416 1 advert, 53.7ms
image 2/9 /home/jovyan/notebooks/test_images/test_2.jpg: 416x416 1 advert, 5.0ms
image 3/9 /home/jovyan/notebooks/test_images/test_3.jpg: 416x320 1 advert, 56.5ms
image 4/9 /home/jovyan/notebooks/test_

# References 

* https://kikaben.com/yolov5-transfer-learning-dogs-cats/